In [ ]:
#DATA VISULALIZATION AND STATISTICS

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv('all_labels.csv')
df.head()

list=df['class'].unique()
print(list)
print(len(list))
count_species = {}
for species in list:
    for x in df['class']:
      if(x==species):
        count_species[species] = count_species.get(species, 0) + 1


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# create and view data
data = pd.DataFrame(count_species.items(),columns=['animals','count'])
data


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Sample DataFrame with 45 animals
np.random.seed(42)  # For reproducibility

# Define colors for bars
colors = plt.cm.viridis(np.linspace(0, 1, len(data)))  # Gradient color map

# Animals to highlight
small_animals = [
    "Aardvark",
    "Aardwolf",
    "Agouti",
    "Coiban_agouti",
    "Collared_peccary",
    "Common_opossum",
    "DikDik",
    "GazelleGrants",
    "GazelleThomsons",
    "HyenaSpotted",
    "Impala",
    "Jackal",
    "Mongoose",
    "Ocelot",
    "Paca",
    "Porcupine",
    "Red_brocket_deer",
    "Red_fox",
    "Red_squirrel",
    "Rodent",
    "Serval",
    "Spiny_rat",
    "Warthog",
    "CattleEgret",
    "GreyBackedFiscal",
    "GuineaFowl",
    "SecretaryBird",
    "SuperbStarling",
    "WattledStarling",
    "WhiteHeadBuffaloWeaver",
    "Tinamou"
]

# Create figure and axis
fig, ax = plt.subplots(figsize=(10, 15))  # Increase height to fit all labels

# Horizontal bar plot
ax.barh(data["animals"], data["count"], color=colors)

# Labels and title
plt.xlabel("Count")
plt.ylabel("Animals")
plt.title("Animal Count Visualization")

# Highlight specific labels in bold red
for label in ax.get_yticklabels():
    if label.get_text() in small_animals:
        label.set_color("red")
        label.set_fontweight("bold")

# Improve layout
plt.gca().invert_yaxis()  # Invert y-axis so highest values appear on top
plt.show()


In [ ]:
#IMAGES DOWNLOADING FROM SNAPSHOT

import json
from collections import Counter

# Path to the metadata file in your drive
metadata_path = '/content/drive/MyDrive/snapshot_safari_2024_metadata.json'

# Load the metadata file
with open(metadata_path, 'r') as f:
    data = json.load(f)

# Count species
species_map = {cat['id']: cat['name'] for cat in data['categories']}
species_count = Counter(species_map[ann['category_id']] for ann in data['annotations'])

# Print counts of all species
for species, count in species_count.items():
    print(f"{species}: {count}")

# Specifically for Impala
print("\nTotal Impala Images:", species_count.get('impala', 0))


In [ ]:
#IMAGES DOWNLOADING FROM WCS

import os
import json
import csv
import requests
from tqdm import tqdm
from collections import defaultdict

# Define species of interest
species_of_interest = {
    "proteles cristata", "ceratotherium simum", "dicerorhinus sumatrensis", "diceros bicornis",
    "lepus saxatilis", "kobus ellipsiprymnus", "chlorocebus pygerythrus", "tinamus sp", "tinamus tao",
    "tinamus major", "tinamus guttatus", "crypturellus erythropus", "odocoileus virginianus",
    "didelphis marsupialis", "didelphis sp", "didelphis virginiana", "didelphis pernigra",
    "didelphis imperfecta", "proechimys sp", "proechimys", "sciurus spadiceus", "sciurus ignitus",
    "sciurus igniventris", "microsciurus flaviventer", "sciurus gramatensis", "funisciurus carruthersi",
    "heliosciurus rufobrachium", "heliosciurus ruwenzorii", "sciurus deppei", "sundasciurus hippurus",
    "callosciurus notatus", "callosciurus erythraeus", "sciurus sp", "funisciurus pyrropus",
    "tragelaphus scriptus", "nasua narica", "nasua nasua", "leopardus pardalis", "dasyprocta punctata",
    "dasyprocta fuliginosa", "dasyprocta leporina", "sus scrofa", "cuniculus paca", "cuniculus taczanowskii",
    "mazama americana", "pecari tajacu"
}

# Load JSON data
json_file = "wcs_20220205_bboxes_with_classes.json"
with open(json_file, "r") as f:
    data = json.load(f)

# Base download directory
download_dir = r"D:\HPE\Dataset\WCS_Images"
os.makedirs(download_dir, exist_ok=True)

# CSV file to store mappings
csv_file = "image_download_mapping.csv"

# Base URL for downloads
base_url = "https://storage.googleapis.com/public-datasets-lila/wcs-unzipped/"

# Load existing mappings to resume downloads
downloaded_images = set()
species_count = defaultdict(int)

if os.path.exists(csv_file):
    with open(csv_file, "r") as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        for row in reader:
            downloaded_images.add(row[0])  # Original file name
            species_name = row[1].split(os.sep)[-2]  # Extract species name
            species_count[species_name] += 1

# Store new mappings
mapping_data = []

# Process images
for image in tqdm(data["images"], desc="Downloading Images"):
    file_name = image["file_name"]  # Example: "humans/0410/1262.jpg"

    if file_name in downloaded_images:
        continue  # Skip already downloaded files

    image_id = image["id"]

    # Get category_id from annotations
    category_id = next((ann["category_id"] for ann in data["annotations"] if ann["image_id"] == image_id), None)
    if category_id is None:
        continue

    # Get species name from categories
    species_name = next((cat["name"] for cat in data["categories"] if cat["id"] == category_id), "unknown")

    # If species is of interest and limit is not reached
    if species_name.lower() in species_of_interest and species_count[species_name] < 2500:
        image_url = base_url + file_name  # Construct full URL

        # Format filename to avoid conflicts
        formatted_name = file_name.replace("/", "_")  # Example: "humans_0410_1262.jpg"
        save_path = os.path.join(download_dir, formatted_name)

        try:
            response = requests.get(image_url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, "wb") as img_file:
                    for chunk in response.iter_content(1024):
                        img_file.write(chunk)

                # Update tracking
                species_count[species_name] += 1
                mapping_data.append([file_name, save_path])

                # Append to CSV immediately to save progress
                with open(csv_file, "a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow([file_name, save_path])

        except Exception as e:
            print(f"Failed to download {image_url}: {e}")

print(f"Download completed. Mapping saved in {csv_file}")


In [ ]:
#AUGMENTATION
import albumentations as A
import cv2
import pandas as pd
import os
import random


meta_df = pd.read_csv('meta_data_13.csv')
output_dir = 'C:/Users/vedit/Pictures/Data_Extracted/images'
os.makedirs(output_dir, exist_ok=True)
augmented_metadata = []


standard_transform = lambda include_noise: A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomGamma(p=0.2),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.3),
    A.GaussNoise(var_limit=(5.0, 10.0), p=0.4) if include_noise else A.NoOp()
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

mild_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.3),
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))


species_counts = meta_df['filename'].drop_duplicates().groupby(meta_df['class']).count()
species_to_augment = species_counts[species_counts < 1600].index.tolist()

def augment_image(image_path, bboxes, labels, species, count):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load {image_path}")
            return

        height, width = image.shape[:2]
        use_mild = any(w * h < 0.02 for _, _, w, h in bboxes)
        use_noise = any(w * h > 0.1 for _, _, w, h in bboxes)

        transform = mild_transform if use_mild else standard_transform(include_noise=use_noise)
        augmented = transform(image=image, bboxes=bboxes, labels=labels)
        augmented_image = augmented['image']
        new_filename = f"{species}_aug_{count}_{os.path.basename(image_path)}"
        new_path = os.path.join(output_dir, new_filename)
        cv2.imwrite(new_path, augmented_image)

        for bbox, label in zip(augmented['bboxes'], augmented['labels']):
            x_c, y_c, w, h = bbox
            if x_c < 0 or y_c < 0 or x_c > 1 or y_c > 1 or w * h < 0.001:
                continue

            augmented_metadata.append({
                'filename': new_filename,
                'width': width,
                'height': height,
                'class': label,
                'x_center': x_c,
                'y_center': y_c,
                'bbox_width': w,
                'bbox_height': h
            })

    except Exception as e:
        print(f"Error processing {image_path}: {e}")


for species in species_to_augment:
    species_data = meta_df[meta_df['class'] == species]
    unique_images = species_data['filename'].unique()
    current_count = len(unique_images)
    augment_count = 0

    while current_count + augment_count < 1600:
        image_name = random.choice(unique_images)
        rows = species_data[species_data['filename'] == image_name]
        boxes = rows[['x_center', 'y_center', 'bbox_width', 'bbox_height']].values.tolist()
        labels = rows['class'].tolist()

        if not any(w * h >= 0.001 for _, _, w, h in boxes):
            continue

        augment_count += 1
        augment_image(image_name, boxes, labels, species, augment_count)


pd.DataFrame(augmented_metadata).to_csv('augmented_meta_data_13.csv', index=False)
print("✅ Done. Saved with conditional Gauss noise for large objects.")


In [ ]:
import pandas as pd
import os

# === CONFIG ===
csv_path = 'metadata15.csv'  # Path to CSV
label_output_dir = 'labels'         # Output dir for .txt label files
yaml_output_path = 'data.yaml'           # Path for YAML config
train_images_path = 'images/train'       # Update as per your YOLO folder structure
val_images_path = 'images/val'           # Optional: if splitting dataset
os.makedirs(label_output_dir, exist_ok=True)

# === LOAD CSV ===
df = pd.read_csv(csv_path)

# === Encode class names to class IDs ===
class_list = sorted(df['class'].unique())
class_to_id = {cls_name: idx for idx, cls_name in enumerate(class_list)}

# === Write YOLO label files ===
for filename, group in df.groupby('filename'):
    txt_filename = os.path.splitext(filename)[0] + '.txt'
    label_file_path = os.path.join(label_output_dir, txt_filename)

    with open(label_file_path, 'w') as f:
        for _, row in group.iterrows():
            class_id = class_to_id[row['class']]
            f.write(f"{class_id} {row['x_center']} {row['y_center']} {row['bbox_width']} {row['bbox_height']}\n")

# === Write data.yaml ===
yaml_lines = [
    f"path: .",
    f"train: {train_images_path}",
    f"val: {val_images_path}",
    f"nc: {len(class_list)}",
    f"names: {class_list}"
]

with open(yaml_output_path, 'w') as f:
    f.write('\n'.join(yaml_lines))

print(f"✅ YOLO labels written to: {label_output_dir}")
print(f"✅ data.yaml written to: {yaml_output_path}")
print("📦 Class map:", class_to_id)


In [ ]:
import pandas as pd

# Load the updated metadata CSV
csv_file = "updated_meta_data_12.csv"
df = pd.read_csv(csv_file)

# Create a list to hold the new rows for meta_data_13
new_rows = []

# Loop through each row in the original CSV
for _, row in df.iterrows():
    filename = row["filename"]
    width = row["width"]
    height = row["height"]
    class_name = row["class"]

    # Normalized values
    x_min = row["x_min"]
    y_min = row["y_min"]
    w = row["w"]
    h = row["h"]

    # Calculate YOLO format values
    x_center = round(x_min + (w / 2), 4)
    y_center = round(y_min + (h / 2), 4)
    bbox_width = round(w, 4)
    bbox_height = round(h, 4)

    # Append the new row to the list
    new_rows.append({
        "filename": filename,
        "width": width,
        "height": height,
        "class": class_name,
        "x_center": x_center,
        "y_center": y_center,
        "bbox_width": bbox_width,
        "bbox_height": bbox_height
    })

# Create a DataFrame for the new CSV
new_df = pd.DataFrame(new_rows)

# Save the new DataFrame to a new CSV file
new_df.to_csv("meta_data_13.csv", index=False)

print("meta_data_13.csv has been created.")


In [ ]:
import pandas as pd
import json
from collections import defaultdict
from os.path import basename, splitext

# Load CSV
csv_path = 'meta_data_9.csv'
df = pd.read_csv(csv_path)

# Load JSON
json_path = 'snapshot_safari_2024_lila_mdv5a.json'
with open(json_path, 'r') as f:
    data = json.load(f)

# Step 1: Prepare JSON bbox mapping using filename (without extension)
json_bboxes = defaultdict(list)
for item in data['images']:
    fname = splitext(basename(item['file']))[0]  # remove extension
    for det in item['detections']:
        bbox = det['bbox']  # [x, y, width, height]
        json_bboxes[fname].append(bbox)

# Step 2: Prepare updated DataFrame
updated_rows = []

# Group by filename (without extension)
df['base_filename'] = df['filename'].apply(lambda x: splitext(basename(x))[0])
filename_groups = df.groupby('base_filename')

for base_fname, group in filename_groups:
    existing_rows = group.copy()
    json_entries = json_bboxes.get(base_fname, [])

    for i, bbox in enumerate(json_entries):
        if i < len(existing_rows):
            # Update existing row
            row = existing_rows.iloc[i].copy()
            row['x_center'], row['y_center'], row['bbox_width'], row['bbox_height'] = bbox
        else:
            # Add new row using data from the first row
            row = existing_rows.iloc[0].copy()
            row['x_center'], row['y_center'], row['bbox_width'], row['bbox_height'] = bbox
        updated_rows.append(row)

    # Add remaining rows if any JSON entry is missing
    if len(json_entries) < len(existing_rows):
        for i in range(len(json_entries), len(existing_rows)):
            updated_rows.append(existing_rows.iloc[i])

# Create new DataFrame
updated_df = pd.DataFrame(updated_rows)
updated_df.drop(columns=['base_filename'], inplace=True)

# Save updated CSV
updated_df.to_csv('meta_data_10.csv', index=False)
print("✅ Updated all bbox values to 'meta_data_9.csv' including additional entries when needed.")


In [ ]:
import pandas as pd
import random
import cv2
import matplotlib.pyplot as plt
import os

# === CONFIG ===
csv_path = 'augmented_meta_data_13.csv'         # Your metadata CSV file
image_base_path = 'C:/Users/vedit/Pictures/augmented'   # Replace with path to your augmented images
num_samples = 100                     # Number of random images to check

# === STEP 1: Load and Filter ===
df = pd.read_csv(csv_path)
df_aug = df[df['filename'].str.contains('animals', case=False, na=False)]
sampled_df = df_aug.sample(n=min(num_samples, len(df_aug)), random_state=42).reset_index(drop=True)

# === STEP 2: Visualize YOLO bboxes ===
def show_image_with_bboxes(img_path, bboxes, labels):
    image = cv2.imread(img_path)
    if image is None:
        print(f"⚠️ Failed to load image: {img_path}")
        return

    height, width = image.shape[:2]

    for bbox, label in zip(bboxes, labels):
        x_center, y_center, w, h = bbox
        x_min = int((x_center - w / 2) * width)
        y_min = int((y_center - h / 2) * height)
        x_max = int((x_center + w / 2) * width)
        y_max = int((y_center + h / 2) * height)

        # Draw bounding box
        color = (0, 255, 0)
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
        cv2.putText(image, str(label), (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)

    # Show image
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# === STEP 3: Loop through sampled images ===
for idx, row in sampled_df.iterrows():
    img_path = os.path.join(image_base_path, row['filename'])

    # Ensure bbox is in YOLO format
    bbox = [row['x_center'], row['y_center'], row['bbox_width'], row['bbox_height']]
    label = row['class_id'] if 'class_id' in row else row['label'] if 'label' in row else 0

    try:
        show_image_with_bboxes(img_path, [bbox], [label])
    except Exception as e:
        print(f"Error showing {row['filename']}: {e}")


In [ ]:
#YOLO416
import os
from ultralytics import YOLO
import pandas as pd
import yaml

data_yaml_path = 'data.yaml'
save_dir = 'results'

os.makedirs(save_dir, exist_ok=True)

model = YOLO("yolov8n.pt")


if __name__ == '__main__':
    train_results = model.train(
        data=data_yaml_path,
        epochs=50,
        batch=4,
        imgsz=416,
        device=0,
        project=save_dir,
        name="yolov8_training",
        save_period=1,
        save_txt=False,
        save_json=False,
        plots=True,
        workers=2,
        val=True
    )


    results_yaml_path = os.path.join(save_dir, "yolov8_training", "results.yaml")

    if os.path.exists(results_yaml_path):
        with open(results_yaml_path, 'r') as f:
            results_data = yaml.safe_load(f)

        metrics_df = pd.DataFrame([results_data])
        metrics_df.to_csv(os.path.join(save_dir, "training_metrics.csv"), index=False)
        print(f"Training metrics saved to {os.path.join(save_dir, 'training_metrics.csv')}")
    else:
        print("Warning: results.yaml not found.")

    print(f"Training completed! Check your results in {save_dir}")


In [ ]:
#YOLO640
import os
from ultralytics import YOLO
import pandas as pd
data_yaml_path = 'data.yaml'
training_name = 'runs/detect'
training_path = training_name


model_path = 'runs/detect/train2_continue/weights/last.pt'
os.makedirs(training_path, exist_ok=True)

model = YOLO(model_path)


if __name__ == '__main__':
    train_results = model.train(
        resume=True,
        epochs=25,
        augment=False,
        batch=4,
        lr0=0.000201,
        optimizer='SGD',
        imgsz=640,
        device=0,
        save_period=1,
        save_txt=False,
        save_json=False,
        plots=True,
        workers=2,
        project='runs/detect',
        name='train2_continue',
        val=True
    )

    results_csv_path = os.path.join(training_path, "results.csv")
    print(f"Training resumed and completed! Check your results in {training_path}")


In [ ]:
#YOLO960
import os
from ultralytics import YOLO
import pandas as pd

data_yaml_path = 'data.yaml'

training_name = 'runs/detect'
training_path = training_name

model_path = 'runs/detect/train3_continue3/weights/last.pt'


os.makedirs(training_path, exist_ok=True)


model = YOLO(model_path)


if __name__ == '__main__':
    train_results = model.train(
        resume=True,
        epochs=50,
        batch=4,
        lr0=0.0005,
        optimizer='SGD',
        imgsz=960,
        device=0,
        save_period=1,
        save_txt=False,
        save_json=False,
        plots=True,
        workers=2,
        project='runs/detect',
        name='train3_continue',
        val=True,
        mosaic=0.0,
        mixup=0.0,
        copy_paste=0.0,
        degrees=0.0,
        shear=0.0,
        perspective=0.0,
        flipud=0.0,
    )

    results_csv_path = os.path.join(training_path, "results.csv")
    print(f"Training resumed and completed! Check your results in {training_path}")


In [ ]:
#STREAMLIT FRONTEND
import streamlit as st
from PIL import Image
import torch
from ultralytics import YOLO
import numpy as np

model = YOLO('/Users/monishp/Downloads/runs/train/experiment_1/weights/best.pt')

st.title("Animal Classification Detection App")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png","JPG"])
if uploaded_file is not None:
    image = Image.open(uploaded_file).convert('RGB')
    st.image(image, caption='Uploaded Image', use_column_width=True)

    # Run YOLO inference
    results = model(image)

    # Show the image with detections
    res_plotted = results[0].plot()  # Plots bounding boxes on image
    st.image(res_plotted, caption='Detected Image', use_column_width=True)